（25分） 实操题1：
计算profile.json（课件数据）中的根据register_time”，不同系统的注册人数，上交代码。

In [1]:
import json
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setMaster('local[*]').setAppName('profile_count')
sc = SparkContext(conf=conf)

In [3]:
rdd_json = sc.textFile('file:///D:/Projects/python_projects/big_data/data/profile.json')
profile_rdd = rdd_json.map(lambda line: json.loads(line))

In [4]:
# 提取字典里面的properties
properties_rdd=profile_rdd.map(lambda x: x['properties'])

In [5]:
properties_rdd.first()

{'os_ver': 'Android OS 8.0.0 / API-26 (HUAWEIFRD-AL00/535(C00))',
 'device_id': 'ce7a9c904768320eadee8dd238550dcb',
 'device_model': 'HUAWEI FRD-AL00',
 'os': 'Android',
 'server_id': 102,
 'login_ip': '222.188.128.144',
 'register_time': '2018-10-16 10:01:45.000',
 'log_type': 'account',
 'connect_id': 4,
 'platform_id': 1,
 'register_ip': '117.63.92.26',
 'channel_id': 104,
 'account': 'UC.aed7566e01ed4fb99be03f980ff4293f',
 'game_id': 1,
 'app_code': 3}

In [6]:
# 提取操作系统与注册时间
def get_os_time(x):
    os=''
    reg_time=''
    for key, val in x.items():
        if key=='os':
            os=val
        if key=='register_time':
            reg_time=val
        yield (os, reg_time)
        

properties_data=properties_rdd.flatMap(get_os_time)    

In [7]:
# 化简日期
def get_register_date(x):
    date = x[1][:10]
    return (x[0], date)


properties_data = properties_data.map(get_register_date)

In [8]:
properties_data.take(10)

[('', ''),
 ('', ''),
 ('', ''),
 ('Android', ''),
 ('Android', ''),
 ('Android', ''),
 ('Android', '2018-10-16'),
 ('Android', '2018-10-16'),
 ('Android', '2018-10-16'),
 ('Android', '2018-10-16')]

In [9]:
# 调换系统与日期的位置
properties_data=properties_data.map(lambda x: (x[1], x[0]))

In [10]:
# 过滤空值
properties_data=properties_data.filter(lambda x: x[1]!='')
properties_data=properties_data.filter(lambda x: x[0]!='')

In [11]:
# 按时间排序
res_time = properties_data.countByValue()
sort_time = sorted(res_time.items(), key=lambda x: x[0], reverse=False)

In [12]:
sort_time

[(('2018-08-24', 'Android'), 12),
 (('2018-08-25', 'Android'), 6),
 (('2018-08-25', 'WindowsEditor'), 6),
 (('2018-08-30', 'Android'), 1884),
 (('2018-08-30', 'Android OS'), 1590),
 (('2018-08-31', 'Android'), 150),
 (('2018-08-31', 'Android OS'), 138),
 (('2018-09-01', 'Android'), 54),
 (('2018-09-01', 'Android OS'), 48),
 (('2018-09-02', 'Android'), 30),
 (('2018-09-02', 'Android OS'), 54),
 (('2018-09-03', 'Android'), 42),
 (('2018-09-03', 'Android OS'), 36),
 (('2018-09-04', 'Android'), 18),
 (('2018-09-04', 'Android OS'), 24),
 (('2018-09-05', 'Android'), 12),
 (('2018-09-05', 'Android OS'), 12),
 (('2018-09-11', 'Android'), 6),
 (('2018-09-26', 'Android'), 18),
 (('2018-09-27', 'Android'), 72),
 (('2018-09-27', 'Android OS'), 24),
 (('2018-09-28', 'Android'), 12),
 (('2018-10-09', 'Android'), 6),
 (('2018-10-11', 'Android OS'), 12),
 (('2018-10-12', 'Android'), 24),
 (('2018-10-12', 'Android OS'), 18),
 (('2018-10-16', 'Android'), 2229),
 (('2018-10-16', 'Android OS'), 48090),
 (